# SQL Project

There is startup that is developing new apps for book lovers. 

There is a database contains data on books, publishers, authors, and customer ratings and reviews of books. 

The purpose of the study: quickly get some data and statistics about the newest books, the largest publisher, the top-rated author and etc. directly from the database to generate a value proposition for a new product in the future. 

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

# some global constats for formatting the output string
BOLD = '\033[1m'
END = '\033[0m'

# DB conection config
db_config = {'user': 'user',             # user name
             'pwd': 'password',          # password
             'host': 'host',
             'port': 1234,               # connection port
             'db': 'db_name'}            # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# function to obtain result of a query
def get_query_result(query):
    return pd.io.sql.read_sql(query, con = engine)

In [2]:
# Let's study the tables by printing the first rows.
# creating query string
get_head_query = 'SELECT * FROM {} LIMIT 5'

# creating list of tables
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

# printing the first few rows of the tables
for table in tables:
    print(BOLD + "The first few rows of the '" + table + "' table:" + END)
    display(pd.read_sql(get_head_query.format(table), engine))
    print()

The first few rows of the 'books' table:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



The first few rows of the 'authors' table:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



The first few rows of the 'publishers' table:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



The first few rows of the 'ratings' table:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



The first few rows of the 'reviews' table:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


__Task1__: Find the number of books released after January 1, 2000.

In [3]:
task1_query = '''SELECT COUNT(*)
                 FROM books
                 WHERE publication_date > '2000-01-01'
              '''

print('The number of books released after January 1, 2000 is' + BOLD, 
      get_query_result(task1_query).values[0][0])

The number of books released after January 1, 2000 is 819


__Task2__: Find the number of user reviews and the average rating for each book.

In [4]:
task2_query = '''SELECT books.*, COUNT(review_id) as reviews_num, AVG(rating) as average_rating
                 FROM 
                     books 
                     LEFT OUTER JOIN ratings ON books.book_id = ratings.book_id
                     LEFT OUTER JOIN reviews ON books.book_id = reviews.book_id      
                 GROUP BY books.book_id
                 ORDER BY books.book_id
              '''
# getting the number of user reviews and the average rating for each book in a table
books_with_reviews_num_and_avg_rating = get_query_result(task2_query)
print('The first few rows of' + BOLD + ' books table with the number of user reviews and the average rating:' + END)
display(books_with_reviews_num_and_avg_rating.head())

The first few rows of books table with the number of user reviews and the average rating:


,book_id,author_id,title,num_pages,publication_date,publisher_id,reviews_num,average_rating
0,1,546,'Salem's Lot,594,2005-11-01,93,6,3.666667
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336,2,2.500000
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135,9,4.666667
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309,4,4.500000
4,5,125,1776,386,2006-07-04,268,24,4.000000


In [5]:
# version 2
task2_query = '''SELECT books.book_id, COUNT(review_id) as reviews_num, AVG(rating) as average_rating
                 FROM 
                     books 
                     LEFT OUTER JOIN ratings ON books.book_id = ratings.book_id
                     LEFT OUTER JOIN reviews ON books.book_id = reviews.book_id      
                 GROUP BY books.book_id
                 ORDER BY reviews_num DESC, average_rating DESC
              '''
# getting the number of user reviews and the average rating for each book in a table
books_with_reviews_num_and_avg_rating = get_query_result(task2_query)
print('The first few rows of table with' + BOLD 
      + ' the number of user reviews and the average rating for each book_id:' + END)
display(books_with_reviews_num_and_avg_rating.head())

The first few rows of table with the number of user reviews and the average rating for each book_id:


,book_id,reviews_num,average_rating
0,948,1120,3.662500
1,750,528,4.125000
2,673,516,3.825581
3,302,492,4.414634
4,299,480,4.287500


__Task3__: Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [6]:
task3_query = '''SELECT publishers.*
                 FROM 
                     publishers 
                     INNER JOIN books ON publishers.publisher_id = books.publisher_id
                 WHERE num_pages > 50      
                 GROUP BY publishers.publisher_id
                 ORDER BY COUNT(book_id) DESC
                 LIMIT 1
              '''

print(BOLD + 'Data on the publisher that has released the greatest number of books with more than 50 pages:' + END)
display(get_query_result(task3_query))

Data on the publisher that has released the greatest number of books with more than 50 pages:


,publisher_id,publisher
0,212,Penguin Books


In [7]:
# version 2
task3_query = '''SELECT publishers.*, COUNT(book_id) as book_num
                 FROM 
                     publishers 
                     INNER JOIN books ON publishers.publisher_id = books.publisher_id
                 WHERE num_pages > 50      
                 GROUP BY publishers.publisher_id
                 ORDER BY COUNT(book_id) DESC
                 LIMIT 1
              '''

print(BOLD + 'Data on the publisher that has released the greatest number of books with more than 50 pages:' + END)
display(get_query_result(task3_query))

Data on the publisher that has released the greatest number of books with more than 50 pages:


,publisher_id,publisher,book_num
0,212,Penguin Books,42


__Task4__: Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [8]:
task4_query = '''SELECT authors.*
                 FROM 
                     authors
                     INNER JOIN books ON authors.author_id = books.author_id
                     INNER JOIN ratings ON books.book_id = ratings.book_id     
                 GROUP BY authors.author_id
                 HAVING COUNT(rating_id) >= 50
                 ORDER BY AVG(rating) DESC
                 LIMIT 1
              '''

print(BOLD + 'Data on the author with the highest average book rating'  + END 
           + ' (look only at books with at least 50 ratings):')
display(get_query_result(task4_query))

Data on the author with the highest average book rating (look only at books with at least 50 ratings):


,author_id,author
0,130,Diana Gabaldon


In [10]:
# version 2
task4_query = '''SELECT authors.*, AVG(rating) as avg_rating, COUNT(rating_id) as rating_num
                 FROM 
                     authors
                     INNER JOIN books ON authors.author_id = books.author_id
                     INNER JOIN ratings ON books.book_id = ratings.book_id     
                 GROUP BY authors.author_id
                 HAVING COUNT(rating_id) >= 50
                 ORDER BY AVG(rating) DESC
                 LIMIT 1
              '''

print(BOLD + 'Data on the author with the highest average book rating'  + END 
           + ' (look only at books with at least 50 ratings):')
display(get_query_result(task4_query))

Data on the author with the highest average book rating (look only at books with at least 50 ratings):


,author_id,author,avg_rating,rating_num
0,130,Diana Gabaldon,4.3,50


__Task5__: Find the average number of text reviews among users who rated more than 50 books.

In [11]:
task5_query = '''SELECT AVG(text_reviews_num)
                 FROM 
                     (SELECT ratings.username, COUNT(DISTINCT review_id) as text_reviews_num
                      FROM 
                      reviews
                      RIGHT OUTER JOIN ratings ON reviews.username = ratings.username         
                      GROUP BY ratings.username
                      HAVING COUNT(DISTINCT ratings.book_id) > 50) as A
              '''

print('The average number of text reviews among users who rated more than 50 books is' + BOLD, 
      get_query_result(task5_query).values[0][0].round(2))

The average number of text reviews among users who rated more than 50 books is 24.33
